In [1]:
from dask.distributed import Client, LocalCluster, worker, progress, wait
import dask
from dask_jobqueue import SLURMCluster

In [2]:
cluster = SLURMCluster()
client=Client(cluster)
cluster.scale(20)
client


Bad key "backend.qt5" on line 5 in
/home/mrayson/.config/matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


Client Scheduler: tcp://146.118.38.44:36232 Dashboard: http://146.118.38.44:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [3]:
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime
import numpy as np
from os import system as sys

import xarray as xr
import pandas as pd
import dask
import glob
from scipy import signal

import matplotlib.pyplot as plt
import matplotlib as mpl

In [5]:
basedir = '/scratch/pawsey0219/ijanekovic/ROMS_CYCLE/outputs/'

In [6]:
roms = xr.open_mfdataset('{}/fwd_ver_7*.nc'.format(basedir),concat_dim='ocean_time',parallel=True)

In [22]:
def write_filtered_var(myzeta, varname, is2d, klayer):    
    for ii,t1 in enumerate(times[0:-1]):
        daysince2000 = (t1-np.datetime64('2000-01-01')).astype(float)*1e-9/86400
        outfile = '../tmp/std_i_{}.nc'.format(int(daysince2000))
        print('\tSaving {} for step {}'.format(varname,t1))
        # This is way quicker when working off the chunked zarr data
        zeta_f_time = myzeta.sel(ocean_time = slice(t1-np.timedelta64(1,'D'), t1+np.timedelta64(5,'D'))).std(axis=0, keepdims=True)
        nc = Dataset(outfile, mode='r+')
        #nc['ocean_time'][0] = daysince2000
        #nc['zeta'][:] = zeta_fp_std[ii,...]
        # 2D variable
        if is2d:
            nc[varname][:] = zeta_f_time
        else:
            # 3D variable
            nc[varname][0,klayer,...] = zeta_f_time
        nc.close()

def filter_roms_2d(roms, varname, cutoff_dt, dt):    
    timevar, yvar, xvar = roms[varname].dims
    arr = roms[varname].chunk({'ocean_time':-1,yvar:10,xvar:10})
    # Compute the filted variable
    arr_filt = xr.apply_ufunc(filt,
                              arr, cutoff_dt, dt,
                              dask='parallelized',
                              #dask='allowed',
                              output_dtypes=[arr.dtype],
                              kwargs={'axis': 0, 'order':8}
                             )
    return arr_filt

def filter_roms_3d(roms, varname, klayer, cutoff_dt, dt):    
    timevar,zvar, yvar, xvar = roms[varname].dims
    # Subset by layer
    s_rho = roms['s_rho'].values
    myroms = roms.sel(s_rho=s_rho[klayer])    
    arr = myroms[varname].chunk({'ocean_time':-1,yvar:10,xvar:10})
    # Compute the filted variable
    arr_filt = xr.apply_ufunc(filt,
                              arr, cutoff_dt, dt,
                              dask='parallelized',
                              #dask='allowed',
                              output_dtypes=[arr.dtype],
                              kwargs={'axis': 0, 'order':8}
                             )
    return arr_filt	

def filt(ytmp, cutoff_dt, dt, btype='low', order=3, axis=0 ):
    """
    Butterworth filter the time series
    Inputs:
        cutoff_dt - cuttoff period [seconds]
        btype - 'low' or 'high' or 'band'
    """
    if not btype == 'band':
        Wn = dt/cutoff_dt
    else:
        Wn = [dt/co for co in cutoff_dt]    
    #(b, a) = signal.butter(order, Wn, btype=btype, analog=0, output='ba', fs=1)    
    #return signal.filtfilt(b, a, ytmp, axis=axis, padlen=0)
    print(ytmp.shape, dt, cutoff_dt)
    sos = signal.butter(order, Wn, btype=btype, analog=0, output='sos', fs=1)    
    return signal.sosfiltfilt(sos, ytmp, axis=axis, padlen=0)


In [8]:
myroms = roms.sel(ocean_time=slice('2020-10-05','2020-12-04'))
varnames = ['zeta','ubar','vbar','temp','salt','u','v']
is2ds = [True, True, True, False, False, False, False]
klayers = 30
times = np.array(pd.date_range('2020-10-09','2020-12-04', freq='4D'))
cutoff_dt = 34*3600.0
# start of the processing
# Save to "std" format file
# Create a file from template
infile = '{}/std_i_template.nc'.format(basedir)
dsin = xr.open_dataset(infile)

In [9]:
for t1 in times[0:-1]:
    daysince2000 = (t1-np.datetime64('2000-01-01'))/np.timedelta64(1, 's')/86400
    outfile = '../tmp/std_i_{}.nc'.format(int(daysince2000))
    dsin.to_netcdf(outfile)
    print(outfile)


../tmp/std_i_7587.nc
../tmp/std_i_7591.nc
../tmp/std_i_7595.nc
../tmp/std_i_7599.nc
../tmp/std_i_7603.nc
../tmp/std_i_7607.nc
../tmp/std_i_7611.nc
../tmp/std_i_7615.nc
../tmp/std_i_7619.nc
../tmp/std_i_7623.nc
../tmp/std_i_7627.nc
../tmp/std_i_7631.nc
../tmp/std_i_7635.nc
../tmp/std_i_7639.nc


In [10]:
myroms = roms.sel(ocean_time=slice('2020-10-05','2020-12-04'))

In [27]:
dt = myroms.ocean_time[2] - myroms.ocean_time[1]
dtsec = dt.values.astype(float)*1e-9    

In [28]:
dt, dtsec

(<xarray.DataArray 'ocean_time' ()>
 array(7200000000000, dtype='timedelta64[ns]'), 7200.0)

In [13]:
zeta_f = filter_roms_2d(myroms, 'zeta', cutoff_dt, dtsec)

(0, 0, 0) 7200.0 122400.0


In [14]:
timevar, yvar, xvar = zeta_f.dims
# Rechunk the data
zeta_fpc = zeta_f.chunk({'ocean_time':1,yvar:-1,xvar:-1})
zeta_fpc = zeta_fpc.persist()        

In [15]:
times[0]

numpy.datetime64('2020-10-09T00:00:00.000000000')

In [16]:
write_filtered_var(zeta_fpc, 'zeta', True, 0)

	Saving zeta for step 2020-10-09T00:00:00.000000000
	Saving zeta for step 2020-10-13T00:00:00.000000000
	Saving zeta for step 2020-10-17T00:00:00.000000000
	Saving zeta for step 2020-10-21T00:00:00.000000000
	Saving zeta for step 2020-10-25T00:00:00.000000000
	Saving zeta for step 2020-10-29T00:00:00.000000000
	Saving zeta for step 2020-11-02T00:00:00.000000000
	Saving zeta for step 2020-11-06T00:00:00.000000000
	Saving zeta for step 2020-11-10T00:00:00.000000000
	Saving zeta for step 2020-11-14T00:00:00.000000000
	Saving zeta for step 2020-11-18T00:00:00.000000000
	Saving zeta for step 2020-11-22T00:00:00.000000000
	Saving zeta for step 2020-11-26T00:00:00.000000000
	Saving zeta for step 2020-11-30T00:00:00.000000000


In [17]:
# The real thing...

In [29]:
def write_filtered_var(myzeta, varname, is2d, klayer, write_time=False):
    
    for ii,t1 in enumerate(times[0:-1]):


        daysince2000 = (t1-np.datetime64('2000-01-01')).astype(float)*1e-9/86400

        outfile = '../tmp/std_i_{}.nc'.format(int(daysince2000))
        print('\tSaving {} for step {}'.format(varname,t1))

        # This is way quicker when working off the chunked zarr data
        zeta_f_time = myzeta.sel(ocean_time = slice(t1-np.timedelta64(1,'D'), t1+np.timedelta64(5,'D'))).std(axis=0, keepdims=True)

        nc = Dataset(outfile, mode='r+')
        if write_time:
            nc['ocean_time'][0] = daysince2000
        #nc['zeta'][:] = zeta_fp_std[ii,...]
        # 2D variable
        if is2d:
            nc[varname][:] = zeta_f_time
        else:
            # 3D variable
            nc[varname][0,klayer,...] = zeta_f_time

        nc.close()


In [31]:
varnames = [
    'zeta',
    'ubar','vbar','temp','salt','u','v']
is2ds = [
    True,
    True, True, False, False, False, False]
klayers = 30

outzarr = '../DATA/tmp_zarr'
# # 

write_time = True

for varname, is2d in zip(varnames, is2ds):
    
    #if varname in ['zeta','ubar','vbar','temp','salt']:
    #    continue
        
    #if varname in ['zeta','temp','salt']:
    #    xvar = 'eta_rho'
    #    yvar = 'xi_rho'
    
    print(72*'#')
    print('Filtering variable: ', varname)
    # Load the variable
    myroms # already done
    
    dt = myroms.ocean_time[2] - myroms.ocean_time[1]
    dtsec = dt.values.astype(float)*1e-9
    
    if is2d:
        zeta_f = filter_roms_2d(myroms, varname, cutoff_dt, dtsec)
        
        timevar, yvar, xvar = zeta_f.dims

        # Rechunk the data
        zeta_fpc = zeta_f.chunk({'ocean_time':1,yvar:-1,xvar:-1})
        
        # Persist the data
        #zeta_fpc = zeta_fpc.persist()
        
        # Try writing and then re-loading the data to ensure calculations work
        print('\tWriting temp data to zarr...')
        xr.Dataset({varname:zeta_fpc}).to_zarr(outzarr,\
                consolidated=True, mode='w', compute=True)
        zeta_fpc = xr.open_zarr(outzarr)[varname]
        
        
        # Write
        write_filtered_var(zeta_fpc, varname, is2d, 0, write_time=write_time)
        write_time = False
    else:
        for klayer in range(klayers):
            print('Filtering layer: ', klayer)
            zeta_f = filter_roms_3d(myroms, varname, klayer, cutoff_dt, dtsec)
            
            timevar, yvar, xvar = zeta_f.dims

             # Rechunk the data
            zeta_fpc = zeta_f.chunk({'ocean_time':1,yvar:-1,xvar:-1})
            #zeta_fpc = zeta_fpc.persist()
            
            # Try writing and then re-loading the data to ensure calculations work
            print('\tWriting temp data to zarr...')
            xr.Dataset({varname:zeta_fpc}).to_zarr(outzarr, \
                    consolidated=True, mode='w', compute=True)
            zeta_fpc = xr.open_zarr(outzarr)[varname]
            
            # Write
            write_filtered_var(zeta_fpc, varname, is2d, klayer)
            
print('Done')
print(72*'#')


########################################################################
Filtering variable:  zeta
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving zeta for step 2020-10-09T00:00:00.000000000
	Saving zeta for step 2020-10-13T00:00:00.000000000
	Saving zeta for step 2020-10-17T00:00:00.000000000
	Saving zeta for step 2020-10-21T00:00:00.000000000
	Saving zeta for step 2020-10-25T00:00:00.000000000
	Saving zeta for step 2020-10-29T00:00:00.000000000
	Saving zeta for step 2020-11-02T00:00:00.000000000
	Saving zeta for step 2020-11-06T00:00:00.000000000
	Saving zeta for step 2020-11-10T00:00:00.000000000
	Saving zeta for step 2020-11-14T00:00:00.000000000
	Saving zeta for step 2020-11-18T00:00:00.000000000
	Saving zeta for step 2020-11-22T00:00:00.000000000
	Saving zeta for step 2020-11-26T00:00:00.000000000
	Saving zeta for step 2020-11-30T00:00:00.000000000
########################################################################
Filtering variable:  ubar
(0, 0, 0) 7200.0 1

	Saving temp for step 2020-11-22T00:00:00.000000000
	Saving temp for step 2020-11-26T00:00:00.000000000
	Saving temp for step 2020-11-30T00:00:00.000000000
Filtering layer:  7
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving temp for step 2020-10-09T00:00:00.000000000
	Saving temp for step 2020-10-13T00:00:00.000000000
	Saving temp for step 2020-10-17T00:00:00.000000000
	Saving temp for step 2020-10-21T00:00:00.000000000
	Saving temp for step 2020-10-25T00:00:00.000000000
	Saving temp for step 2020-10-29T00:00:00.000000000
	Saving temp for step 2020-11-02T00:00:00.000000000
	Saving temp for step 2020-11-06T00:00:00.000000000
	Saving temp for step 2020-11-10T00:00:00.000000000
	Saving temp for step 2020-11-14T00:00:00.000000000
	Saving temp for step 2020-11-18T00:00:00.000000000
	Saving temp for step 2020-11-22T00:00:00.000000000
	Saving temp for step 2020-11-26T00:00:00.000000000
	Saving temp for step 2020-11-30T00:00:00.000000000
Filtering layer:  8
(0, 0, 0) 7200.0 122

Filtering layer:  17
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving temp for step 2020-10-09T00:00:00.000000000
	Saving temp for step 2020-10-13T00:00:00.000000000
	Saving temp for step 2020-10-17T00:00:00.000000000
	Saving temp for step 2020-10-21T00:00:00.000000000
	Saving temp for step 2020-10-25T00:00:00.000000000
	Saving temp for step 2020-10-29T00:00:00.000000000
	Saving temp for step 2020-11-02T00:00:00.000000000
	Saving temp for step 2020-11-06T00:00:00.000000000
	Saving temp for step 2020-11-10T00:00:00.000000000
	Saving temp for step 2020-11-14T00:00:00.000000000
	Saving temp for step 2020-11-18T00:00:00.000000000
	Saving temp for step 2020-11-22T00:00:00.000000000
	Saving temp for step 2020-11-26T00:00:00.000000000
	Saving temp for step 2020-11-30T00:00:00.000000000
Filtering layer:  18
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving temp for step 2020-10-09T00:00:00.000000000
	Saving temp for step 2020-10-13T00:00:00.000000000
	Saving temp f

	Saving temp for step 2020-10-17T00:00:00.000000000
	Saving temp for step 2020-10-21T00:00:00.000000000
	Saving temp for step 2020-10-25T00:00:00.000000000
	Saving temp for step 2020-10-29T00:00:00.000000000
	Saving temp for step 2020-11-02T00:00:00.000000000
	Saving temp for step 2020-11-06T00:00:00.000000000
	Saving temp for step 2020-11-10T00:00:00.000000000
	Saving temp for step 2020-11-14T00:00:00.000000000
	Saving temp for step 2020-11-18T00:00:00.000000000
	Saving temp for step 2020-11-22T00:00:00.000000000
	Saving temp for step 2020-11-26T00:00:00.000000000
	Saving temp for step 2020-11-30T00:00:00.000000000
Filtering layer:  28
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving temp for step 2020-10-09T00:00:00.000000000
	Saving temp for step 2020-10-13T00:00:00.000000000
	Saving temp for step 2020-10-17T00:00:00.000000000
	Saving temp for step 2020-10-21T00:00:00.000000000
	Saving temp for step 2020-10-25T00:00:00.000000000
	Saving temp for step 2020-10-29T00:00:

	Saving salt for step 2020-10-21T00:00:00.000000000
	Saving salt for step 2020-10-25T00:00:00.000000000
	Saving salt for step 2020-10-29T00:00:00.000000000
	Saving salt for step 2020-11-02T00:00:00.000000000
	Saving salt for step 2020-11-06T00:00:00.000000000
	Saving salt for step 2020-11-10T00:00:00.000000000
	Saving salt for step 2020-11-14T00:00:00.000000000
	Saving salt for step 2020-11-18T00:00:00.000000000
	Saving salt for step 2020-11-22T00:00:00.000000000
	Saving salt for step 2020-11-26T00:00:00.000000000
	Saving salt for step 2020-11-30T00:00:00.000000000
Filtering layer:  8
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving salt for step 2020-10-09T00:00:00.000000000
	Saving salt for step 2020-10-13T00:00:00.000000000
	Saving salt for step 2020-10-17T00:00:00.000000000
	Saving salt for step 2020-10-21T00:00:00.000000000
	Saving salt for step 2020-10-25T00:00:00.000000000
	Saving salt for step 2020-10-29T00:00:00.000000000
	Saving salt for step 2020-11-02T00:00:0

...

	Saving u for step 2020-10-17T00:00:00.000000000
	Saving u for step 2020-10-21T00:00:00.000000000
	Saving u for step 2020-10-25T00:00:00.000000000
	Saving u for step 2020-10-29T00:00:00.000000000
	Saving u for step 2020-11-02T00:00:00.000000000
	Saving u for step 2020-11-06T00:00:00.000000000
	Saving u for step 2020-11-10T00:00:00.000000000
	Saving u for step 2020-11-14T00:00:00.000000000
	Saving u for step 2020-11-18T00:00:00.000000000
	Saving u for step 2020-11-22T00:00:00.000000000
	Saving u for step 2020-11-26T00:00:00.000000000
	Saving u for step 2020-11-30T00:00:00.000000000
Filtering layer:  9
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving u for step 2020-10-09T00:00:00.000000000
	Saving u for step 2020-10-13T00:00:00.000000000
	Saving u for step 2020-10-17T00:00:00.000000000
	Saving u for step 2020-10-21T00:00:00.000000000
	Saving u for step 2020-10-25T00:00:00.000000000
	Saving u for step 2020-10-29T00:00:00.000000000
	Saving u for step 2020-11-02T00:00:00.000

Filtering layer:  19
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving u for step 2020-10-09T00:00:00.000000000
	Saving u for step 2020-10-13T00:00:00.000000000
	Saving u for step 2020-10-17T00:00:00.000000000
	Saving u for step 2020-10-21T00:00:00.000000000
	Saving u for step 2020-10-25T00:00:00.000000000
	Saving u for step 2020-10-29T00:00:00.000000000
	Saving u for step 2020-11-02T00:00:00.000000000
	Saving u for step 2020-11-06T00:00:00.000000000
	Saving u for step 2020-11-10T00:00:00.000000000
	Saving u for step 2020-11-14T00:00:00.000000000
	Saving u for step 2020-11-18T00:00:00.000000000
	Saving u for step 2020-11-22T00:00:00.000000000
	Saving u for step 2020-11-26T00:00:00.000000000
	Saving u for step 2020-11-30T00:00:00.000000000
Filtering layer:  20
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving u for step 2020-10-09T00:00:00.000000000
	Saving u for step 2020-10-13T00:00:00.000000000
	Saving u for step 2020-10-17T00:00:00.000000000
	Saving u for

	Saving u for step 2020-11-18T00:00:00.000000000
	Saving u for step 2020-11-22T00:00:00.000000000
	Saving u for step 2020-11-26T00:00:00.000000000
	Saving u for step 2020-11-30T00:00:00.000000000
########################################################################
Filtering variable:  v
Filtering layer:  0
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving v for step 2020-10-09T00:00:00.000000000
	Saving v for step 2020-10-13T00:00:00.000000000
	Saving v for step 2020-10-17T00:00:00.000000000
	Saving v for step 2020-10-21T00:00:00.000000000
	Saving v for step 2020-10-25T00:00:00.000000000
	Saving v for step 2020-10-29T00:00:00.000000000
	Saving v for step 2020-11-02T00:00:00.000000000
	Saving v for step 2020-11-06T00:00:00.000000000
	Saving v for step 2020-11-10T00:00:00.000000000
	Saving v for step 2020-11-14T00:00:00.000000000
	Saving v for step 2020-11-18T00:00:00.000000000
	Saving v for step 2020-11-22T00:00:00.000000000
	Saving v for step 2020-11-26T00:00:00.00000

	Saving v for step 2020-10-29T00:00:00.000000000
	Saving v for step 2020-11-02T00:00:00.000000000
	Saving v for step 2020-11-06T00:00:00.000000000
	Saving v for step 2020-11-10T00:00:00.000000000
	Saving v for step 2020-11-14T00:00:00.000000000
	Saving v for step 2020-11-18T00:00:00.000000000
	Saving v for step 2020-11-22T00:00:00.000000000
	Saving v for step 2020-11-26T00:00:00.000000000
	Saving v for step 2020-11-30T00:00:00.000000000
Filtering layer:  11
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving v for step 2020-10-09T00:00:00.000000000
	Saving v for step 2020-10-13T00:00:00.000000000
	Saving v for step 2020-10-17T00:00:00.000000000
	Saving v for step 2020-10-21T00:00:00.000000000
	Saving v for step 2020-10-25T00:00:00.000000000
	Saving v for step 2020-10-29T00:00:00.000000000
	Saving v for step 2020-11-02T00:00:00.000000000
	Saving v for step 2020-11-06T00:00:00.000000000
	Saving v for step 2020-11-10T00:00:00.000000000
	Saving v for step 2020-11-14T00:00:00.00

	Saving v for step 2020-10-13T00:00:00.000000000
	Saving v for step 2020-10-17T00:00:00.000000000
	Saving v for step 2020-10-21T00:00:00.000000000
	Saving v for step 2020-10-25T00:00:00.000000000
	Saving v for step 2020-10-29T00:00:00.000000000
	Saving v for step 2020-11-02T00:00:00.000000000
	Saving v for step 2020-11-06T00:00:00.000000000
	Saving v for step 2020-11-10T00:00:00.000000000
	Saving v for step 2020-11-14T00:00:00.000000000
	Saving v for step 2020-11-18T00:00:00.000000000
	Saving v for step 2020-11-22T00:00:00.000000000
	Saving v for step 2020-11-26T00:00:00.000000000
	Saving v for step 2020-11-30T00:00:00.000000000
Filtering layer:  22
(0, 0, 0) 7200.0 122400.0
	Writing temp data to zarr...
	Saving v for step 2020-10-09T00:00:00.000000000
	Saving v for step 2020-10-13T00:00:00.000000000
	Saving v for step 2020-10-17T00:00:00.000000000
	Saving v for step 2020-10-21T00:00:00.000000000
	Saving v for step 2020-10-25T00:00:00.000000000
	Saving v for step 2020-10-29T00:00:00.00

In [24]:
cutoff_dt/dtsec


/home/mrayson/group/mrayson/miniconda3/envs/soda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in double_scalars
  """Entry point for launching an IPython kernel.


inf

In [25]:
dtsec

0.0